# Task
Unzip `archive.zip`, load the main CSV dataset into a Pandas DataFrame, perform an initial exploration by showing the first 5 rows and checking for null values, and then generate a bar chart to visualize the balance of sentiment categories.

## Descompactar Arquivo

### Subtask:
Descompactar o arquivo 'archive.zip' para acessar os datasets.


## Análise de Sentimento com o Modelo Treinado

### Subtask:
Criar uma função `analisar_mercado(texto)` que utiliza o modelo FinBERT para prever o sentimento de uma manchete e testá-la com um exemplo.

**Reasoning**:
To create the `analisar_mercado` function, I will use the previously loaded `tokenizer` and `model`. The function will tokenize the input text, move the tensors to the GPU if available, get predictions from the model, apply softmax to convert logits to probabilities, and then map the predicted class index to a sentiment label (Negative or Positive) based on the `num_labels` configuration. Finally, I will test the function with the given headline.

In [ ]:
import torch

def analisar_mercado(texto):
    """
    Analisa o sentimento de uma manchete de notícia usando o modelo FinBERT treinado.

    Args:
        texto (str): A manchete de notícia a ser analisada.

    Returns:
        tuple: Uma tupla contendo o rótulo de sentimento (string) e a probabilidade (float).
    """
    # Tokenizar a manchete
    inputs = tokenizer(texto, return_tensors='pt', truncation=True, padding=True)

    # Mover inputs para o dispositivo (GPU se disponível)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model.to(device)

    # Fazer a previsão
    with torch.no_grad():
        outputs = model(**inputs)

    # Obter logits e calcular probabilidades
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=1)
    predicted_class_id = torch.argmax(probabilities, dim=1).item()

    # Mapear o ID da classe para o rótulo de sentimento
    # Baseado em como mapeamos -1 para 0 e 1 para 1 no pré-processamento
    sentiment_labels = {0: 'Negative', 1: 'Positive'}
    sentiment_label = sentiment_labels.get(predicted_class_id, 'Unknown')
    confidence = probabilities[0, predicted_class_id].item()

    return sentiment_label, confidence

# Testar a função com a manchete fornecida
test_headline = "Apple stock price plunged 5% after the new quarterly earnings report showed massive disappointment."
sentiment, confidence = analisar_mercado(test_headline)

print(f"Manchete: '{test_headline}'")
print(f"Sentimento Previsto: {sentiment}")
print(f"Confiança do Modelo: {confidence:.4f}")

**Reasoning**:
To extract the contents of 'archive.zip', I will execute the `!unzip archive.zip` command.



In [ ]:
!unzip archive.zip

## Carregar Dataset Principal

### Subtask:
Carregar o arquivo CSV principal descompactado em um DataFrame do Pandas.


**Reasoning**:
To load the main CSV dataset into a Pandas DataFrame, I will first import the pandas library and then use `pd.read_csv()` to load 'stock_data.csv' into a DataFrame named `df`.



In [ ]:
import pandas as pd
df = pd.read_csv('stock_data.csv')
print("Dataset loaded successfully.")
df.head()

## Exploração Inicial do DataFrame

### Subtask:
Exibir as 5 primeiras linhas do DataFrame e verificar a presença de valores nulos.


**Reasoning**:
To fulfill the subtask of checking for null values, I will use the `.isnull().sum()` method on the DataFrame `df` to get a count of missing values per column. This will provide an overview of data completeness.



In [ ]:
print("Checking for null values in the DataFrame:")
print(df.isnull().sum())

## Análise de Balanceamento de Sentimento

### Subtask:
Gerar um gráfico de barras para visualizar a contagem de cada categoria de 'Sentiment' e analisar o balanceamento dos dados.


## Summary:

### Data Analysis Key Findings
*   The `archive.zip` file was successfully unzipped, and the `stock_data.csv` file was extracted.
*   The `stock_data.csv` dataset was loaded into a Pandas DataFrame for further analysis.
*   An initial check for null values confirmed that both the 'Text' and 'Sentiment' columns in the DataFrame contain no missing values.

### Insights or Next Steps
*   The dataset is clean and ready for sentiment analysis without requiring handling of missing values in the key 'Text' and 'Sentiment' columns.
*   The next crucial step is to generate a bar chart to visualize the distribution of sentiment categories to understand the dataset's balance, as this was the final part of the initial task setup.


# Task
To proceed with the sentiment analysis, the next step is to verify the unique sentiment classes in the dataset, map them to a consistent integer format if necessary, and then display this mapping along with the distribution of each class to ensure proper preparation for model training.

## Verificação e Mapeamento de Classes de Sentimento

### Subtask:
Verificar os valores únicos da coluna 'Sentiment', mapeá-los para inteiros (0, 1 ou 0, 1, 2) conforme necessário e imprimir o mapeamento.


**Reasoning**:
To verify and map the 'Sentiment' column, I will first identify and print the unique values to understand the current sentiment categories. Then, I will create a mapping based on the instructions to convert '-1' to '0' and keep '1' as '1', and apply this mapping to the 'Sentiment' column. Finally, I will display the value counts of the updated 'Sentiment' column to confirm the new distribution.



In [ ]:
print("Unique values in 'Sentiment' before mapping:", df['Sentiment'].unique())

# Check unique values and apply mapping if necessary
if set(df['Sentiment'].unique()) == {-1, 1}:
    sentiment_mapping = {-1: 0, 1: 1}
    df['Sentiment'] = df['Sentiment'].map(sentiment_mapping)
    print("Sentiment mapped: -1 to 0, 1 to 1")
    print("Mapping applied:", sentiment_mapping)
else:
    print("No specific mapping applied as unique sentiment values are not exactly {-1, 1}.")

print("\nSentiment class distribution after mapping:")
print(df['Sentiment'].value_counts())

## Carregamento do Modelo e Tokenizer FinBERT

### Subtask:
Carregar o modelo 'ProsusAI/finbert' e seu tokenizer correspondente, configurando 'num_labels' com base no número de classes de sentimento.


**Reasoning**:
To load the FinBERT model and tokenizer, I will import the necessary classes from the `transformers` library, determine the number of unique sentiment classes from the 'Sentiment' column, and then use these to load the pre-trained tokenizer and model.



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Determine the number of unique sentiment classes
num_labels = df['Sentiment'].nunique()
print(f"Number of sentiment classes: {num_labels}")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
print("FinBERT tokenizer loaded successfully.")

# Load the model, configuring num_labels
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=num_labels)
print("FinBERT model loaded successfully with custom num_labels.")

**Reasoning**:
The previous code failed with a `RuntimeError` due to a size mismatch in the classification layer of the FinBERT model. The pre-trained FinBERT model expects 3 output labels (positive, negative, neutral), but the current task requires only 2 (mapped to 0 and 1). To resolve this, I need to instruct `from_pretrained` to ignore size mismatches in the classification head, allowing it to be reinitialized for the correct number of labels.



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Determine the number of unique sentiment classes
num_labels = df['Sentiment'].nunique()
print(f"Number of sentiment classes: {num_labels}")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
print("FinBERT tokenizer loaded successfully.")

# Load the model, configuring num_labels and ignoring mismatched sizes for the classifier head
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", num_labels=num_labels, ignore_mismatched_sizes=True)
print("FinBERT model loaded successfully with custom num_labels and mismatched sizes ignored.")

**Reasoning**:
The previous code successfully loaded the FinBERT model and tokenizer. However, the initial task included generating a bar chart to visualize the balance of sentiment categories, which has not yet been done. Visualizing the sentiment distribution is crucial for understanding the dataset's balance before proceeding with model training or evaluation.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the value counts of the 'Sentiment' column
sentiment_counts = df['Sentiment'].value_counts()

# Create a bar chart
plt.figure(figsize=(8, 6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette="viridis")
plt.title('Distribution of Sentiment Categories')
plt.xlabel('Sentiment (0: Negative, 1: Positive)')
plt.ylabel('Number of Samples')
plt.xticks(ticks=[0, 1], labels=['Negative', 'Positive'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print("Bar chart showing sentiment distribution generated.")

**Reasoning**:
The previous code generated a `FutureWarning` from `seaborn` regarding the `palette` argument without `hue`. To fix this and comply with future `seaborn` versions, I will explicitly set `hue` to the `x` variable and add `legend=False` as suggested by the warning.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the value counts of the 'Sentiment' column
sentiment_counts = df['Sentiment'].value_counts()

# Create a bar chart
plt.figure(figsize=(8, 6))
sns.barplot(x=sentiment_counts.index, y=sentiment_counts.values, palette="viridis", hue=sentiment_counts.index, legend=False)
plt.title('Distribution of Sentiment Categories')
plt.xlabel('Sentiment (0: Negative, 1: Positive)')
plt.ylabel('Number of Samples')
plt.xticks(ticks=[0, 1], labels=['Negative', 'Positive'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

print("Bar chart showing sentiment distribution generated.")

## Tokenização dos Dados

### Subtask:
Aplicar o tokenizer do FinBERT aos dados textuais do DataFrame para prepará-los para o modelo.


**Reasoning**:
To apply the FinBERT tokenizer to the textual data, I will first import the `Dataset` class, create a Hugging Face Dataset from the DataFrame, define a tokenization function that truncates sequences, and then apply this function using the `.map()` method.



In [ ]:
from datasets import Dataset

# Convert the pandas DataFrame to a Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df, split='train')
print("Hugging Face Dataset created successfully.")

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples['Text'], truncation=True, max_length=512)

# Apply the tokenization function to the dataset
tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, remove_columns=['Text'])
print("Dataset tokenized successfully. Example of tokenized input:")
print(tokenized_datasets[0])

## Divisão dos Dados em Treino e Teste

### Subtask:
Dividir os dados tokenizados em conjuntos de treino (80%) e teste (20%) usando `stratify` na coluna de 'Sentiment' para manter o balanceamento das classes.


**Reasoning**:
To split the tokenized dataset into training and testing sets while maintaining the sentiment class balance, I will use the built-in `train_test_split` method of the Hugging Face `Dataset` object, specifying a `test_size` of 0.2 and stratifying by the 'Sentiment' column.



In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
splitted_datasets = tokenized_datasets.train_test_split(test_size=0.2, stratify_by_column='Sentiment')

train_dataset = splitted_datasets['train']
test_dataset = splitted_datasets['test']

print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("Data split into training and test sets successfully.")

**Reasoning**:
The previous code failed because the 'Sentiment' column in the Hugging Face Dataset was not of type `ClassLabel`, which is required for stratification. To fix this, I need to first cast the 'Sentiment' column to `ClassLabel` with the appropriate number of classes before performing the `train_test_split`.



In [ ]:
from sklearn.model_selection import train_test_split
from datasets import ClassLabel, Features, Value

# Define the features with ClassLabel for 'Sentiment'
features = Features({
    'input_ids': Value(dtype='int32'),
    'token_type_ids': Value(dtype='int32'),
    'attention_mask': Value(dtype='int32'),
    'Sentiment': ClassLabel(num_classes=num_labels, names=['Negative', 'Positive'])
})

# Cast the dataset to include the ClassLabel for 'Sentiment'
tokenized_datasets = tokenized_datasets.cast(features)
print("Sentiment column cast to ClassLabel type.")

# Split the dataset into training and testing sets
splitted_datasets = tokenized_datasets.train_test_split(test_size=0.2, stratify_by_column='Sentiment')

train_dataset = splitted_datasets['train']
test_dataset = splitted_datasets['test']

print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("Data split into training and test sets successfully.")

**Reasoning**:
The previous error 'TypeError: Couldn't cast array of type list<item: int32> to int32' occurred because the `input_ids`, `token_type_ids`, and `attention_mask` columns, which are sequences of integers, were incorrectly defined as single `Value(dtype='int32')` in the `Features` object. To fix this, these columns must be defined as `Sequence(Value(dtype='int32'))` to correctly represent arrays or lists of integers.



In [ ]:
from sklearn.model_selection import train_test_split
from datasets import ClassLabel, Features, Value, Sequence

# Define the features with ClassLabel for 'Sentiment' and Sequence for tokenized inputs
features = Features({
    'input_ids': Sequence(feature=Value(dtype='int32')),
    'token_type_ids': Sequence(feature=Value(dtype='int32')),
    'attention_mask': Sequence(feature=Value(dtype='int32')),
    'Sentiment': ClassLabel(num_classes=num_labels, names=['Negative', 'Positive'])
})

# Cast the dataset to include the ClassLabel for 'Sentiment'
tokenized_datasets = tokenized_datasets.cast(features)
print("Sentiment column and tokenized inputs cast to correct types.")

# Split the dataset into training and testing sets
splitted_datasets = tokenized_datasets.train_test_split(test_size=0.2, stratify_by_column='Sentiment')

train_dataset = splitted_datasets['train']
test_dataset = splitted_datasets['test']

print(f"Training dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")
print("Data split into training and test sets successfully.")

## Configuração do Treinador (Trainer)

### Subtask:
Configurar o objeto `Trainer` do Hugging Face Transformers com `num_train_epochs=3`, `per_device_train_batch_size=16`, e definir as métricas de acurácia e F1-score.


**Reasoning**:
To configure the Hugging Face Trainer, I need to import `TrainingArguments` and `Trainer`, define a `compute_metrics` function using `accuracy_score` and `f1_score`, initialize `TrainingArguments` with the specified parameters, and then instantiate the `Trainer`.



In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # Using weighted for potential class imbalance
    return {"accuracy": accuracy, "f1_score": f1}

# Initialize TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16, # Added to match train batch size as per instructions
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # Use test_dataset for evaluation as per common practice
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Trainer configured successfully.")

**Reasoning**:
The previous code failed because `evaluation_strategy` is an outdated argument for `TrainingArguments`. It has been renamed to `eval_strategy` in newer versions of the `transformers` library. I will update the argument name to `eval_strategy` to resolve this `TypeError`.



In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # Using weighted for potential class imbalance
    return {"accuracy": accuracy, "f1_score": f1}

# Initialize TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16, # Added to match train batch size as per instructions
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='epoch', # Corrected from evaluation_strategy
    save_strategy='epoch'
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # Use test_dataset for evaluation as per common practice
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

print("Trainer configured successfully.")

**Reasoning**:
The previous code generated a `FutureWarning` indicating that the `tokenizer` argument in `Trainer.__init__` is deprecated and should be replaced with `processing_class`. To comply with future versions of the `transformers` library and eliminate the warning, I will update the `Trainer` instantiation to use `processing_class=tokenizer`.



In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # Using weighted for potential class imbalance
    return {"accuracy": accuracy, "f1_score": f1}

# Initialize TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16, # Added to match train batch size as per instructions
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='epoch', # Corrected from evaluation_strategy
    save_strategy='epoch'
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # Use test_dataset for evaluation as per common practice
    processing_class=tokenizer, # Corrected from tokenizer=tokenizer
    compute_metrics=compute_metrics
)

print("Trainer configured successfully.")

## Execução do Treinamento

### Subtask:
Iniciar o treinamento do modelo usando o `Trainer`, garantindo que a GPU (cuda) seja utilizada.


**Reasoning**:
To initiate the training process as per the subtask, I will first verify GPU availability and then call the `train()` method on the configured `trainer` object.



In [ ]:
import torch

# Check for GPU availability
if torch.cuda.is_available():
    print("GPU (cuda) is available. Training will utilize GPU.")
else:
    print("GPU (cuda) is not available. Training will run on CPU.")

# Start training the model
print("Starting model training...")
trainer.train()
print("Model training completed.")

# Task
Ajustar a configuração de treinamento para desativar o registro no Weights & Biases (W&B) e, em seguida, re-executar o treinamento do modelo FinBERT para análise de sentimento. Por fim, forneça um resumo do processo de treinamento e das métricas de desempenho obtidas.

## Ajustar Configuração do Trainer

### Subtask:
Modificar a inicialização do `TrainingArguments` para incluir `report_to='none'`, desativando o registro no Weights & Biases (W&B) e prevenindo o prompt de login.


**Reasoning**:
The previous training attempt failed because of a Weights & Biases login prompt. To disable this, I need to add `report_to='none'` to the `TrainingArguments` initialization as specified in the subtask. I will regenerate the entire code block for configuring the Trainer to incorporate this change.



In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # Using weighted for potential class imbalance
    return {"accuracy": accuracy, "f1_score": f1}

# Initialize TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    report_to='none' # Added to disable Weights & Biases logging
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

print("Trainer configured successfully with W&B logging disabled.")

**Reasoning**:
The previous step successfully reconfigured the Trainer to disable Weights & Biases logging. The next step, as per the main task, is to re-execute the training of the FinBERT model to ensure it runs without interruption.



In [ ]:
import torch

# Check for GPU availability
if torch.cuda.is_available():
    print("GPU (cuda) is available. Training will utilize GPU.")
else:
    print("GPU (cuda) is not available. Training will run on CPU.")

# Start training the model
print("Starting model training...")
trainer.train()
print("Model training completed.")

**Reasoning**:
The training failed because the model expected a 'labels' column for loss computation, but the dataset provided 'Sentiment'. I need to rename the 'Sentiment' column to 'labels' in both the training and test datasets.



In [ ]:
import torch

# Rename the 'Sentiment' column to 'labels' in both train and test datasets
# The Trainer expects the labels column to be named 'labels' by default.
train_dataset = train_dataset.rename_column("Sentiment", "labels")
test_dataset = test_dataset.rename_column("Sentiment", "labels")

print("Sentiment column renamed to labels in train and test datasets.")

# Check for GPU availability
if torch.cuda.is_available():
    print("GPU (cuda) is available. Training will utilize GPU.")
else:
    print("GPU (cuda) is not available. Training will run on CPU.")

# Start training the model
print("Starting model training...")
trainer.train()
print("Model training completed.")

**Reasoning**:
The previous training attempt failed because the `trainer` object was initialized before the 'Sentiment' column in the datasets was renamed to 'labels'. To fix this, I need to re-instantiate the `Trainer` object using the `train_dataset` and `test_dataset` which now correctly have the 'labels' column for loss computation.



In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # Using weighted for potential class imbalance
    return {"accuracy": accuracy, "f1_score": f1}

# Initialize TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy='epoch',
    save_strategy='epoch',
    report_to='none' # Added to disable Weights & Biases logging
)

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)

print("Trainer re-configured successfully with W&B logging disabled and updated datasets.")

**Reasoning**:
The previous steps successfully re-configured the Trainer with the `labels` column and disabled W&B logging. Now, I need to re-initiate the training process to ensure the model trains correctly without errors, as specified by the task.



In [ ]:
import torch

# Check for GPU availability
if torch.cuda.is_available():
    print("GPU (cuda) is available. Training will utilize GPU.")
else:
    print("GPU (cuda) is not available. Training will run on CPU.")

# Start training the model
print("Starting model training...")
trainer.train()
print("Model training completed.")

**Reasoning**:
The model training has been successfully completed. Now, I will evaluate the trained model on the test dataset to get the performance metrics and then summarize the training process and the obtained metrics, as per the main task instructions.



In [ ]:
print("Evaluating the trained model on the test dataset...")
eval_results = trainer.evaluate()

print("Training process and evaluation metrics:")
print(f"Epochs trained: {eval_results['epoch']}")
print(f"Evaluation Loss: {eval_results['eval_loss']:.4f}")
print(f"Evaluation Accuracy: {eval_results['eval_accuracy']:.4f}")
print(f"Evaluation F1-Score (weighted): {eval_results['eval_f1_score']:.4f}")
print("\nSummary of Training:\n")
print(f"Model training was successfully completed over {training_args.num_train_epochs} epochs.")
print(f"The model was trained with a batch size of {training_args.per_device_train_batch_size} on GPU (if available).")
print("Weights & Biases logging was disabled to prevent interruptions.")

## Final Task

### Subtask:
Fornecer um resumo do processo de treinamento do modelo FinBERT para análise de sentimento, incluindo as métricas de desempenho obtidas.


## Summary:

### Q&A
The FinBERT model training process for sentiment analysis involved adjusting the `TrainingArguments` to disable Weights & Biases (W&B) logging, resolving a dataset column mismatch, and then training the model. The final performance metrics obtained were an Evaluation Loss of 0.5413, an Evaluation Accuracy of 0.8179, and an Evaluation F1-Score (weighted) of 0.8169 after 3 epochs of training.

### Data Analysis Key Findings
*   The `report_to='none'` parameter was successfully integrated into `TrainingArguments`, disabling W&B logging and preventing prior interruptions.
*   An initial `ValueError` was encountered because the `Trainer` expected a 'labels' column, while the dataset used 'Sentiment'; this was rectified by renaming the 'Sentiment' column to 'labels' in both training and test datasets.
*   The `Trainer` required re-instantiation after the dataset column renaming for the changes to be recognized.
*   After these adjustments, the FinBERT model was successfully trained for 3 epochs using GPU resources.
*   The trained model achieved an Evaluation Loss of 0.5413, an Evaluation Accuracy of 0.8179, and an Evaluation F1-Score (weighted) of 0.8169 on the test dataset.

### Insights or Next Steps
*   Ensuring dataset column names (e.g., 'labels') align with the expectations of the `Trainer` is crucial for successful model training.
*   To potentially enhance model performance, further hyperparameter tuning (e.g., adjusting `num_train_epochs`, learning rate, or `weight_decay`) could be explored.


# Task
## Gerar Previsões no Conjunto de Teste

### Subtask:
Utilizar o `trainer` treinado para gerar previsões (logits) no `test_dataset`.

## Gerar Previsões no Conjunto de Teste

### Subtask:
Utilizar o `trainer` treinado para gerar previsões (logits) no `test_dataset`.


**Reasoning**:
To generate predictions on the `test_dataset` using the trained `trainer`, I will use the `trainer.predict()` method, which will output the logits.



In [ ]:
print("Generating predictions on the test dataset...")
predictions = trainer.predict(test_dataset)
print("Predictions generated successfully.")
# The 'predictions' object will contain logits, and potentially labels and metrics if available.

## Gerar e Imprimir Relatório de Classificação

### Subtask:
Extrair os rótulos verdadeiros e as previsões do modelo e usar `sklearn.metrics.classification_report` para imprimir a precisão, recall e F1-score para cada classe.


**Reasoning**:
To generate and print the classification report, I will import `classification_report` from `sklearn.metrics`, extract the true labels from `predictions.label_ids` and predicted labels from `predictions.predictions` using `np.argmax`, and then print the report.



In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Extract true labels
true_labels = predictions.label_ids

# Extract predicted labels from logits
predicted_labels = np.argmax(predictions.predictions, axis=-1)

# Print the classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels, target_names=['Negative', 'Positive']))

## Gerar e Plotar Matriz de Confusão

### Subtask:
Calcular a matriz de confusão usando `sklearn.metrics.confusion_matrix` e plotá-la visualmente com `seaborn.heatmap` para identificar onde o modelo cometeu erros de classificação.


**Reasoning**:
To calculate and plot the confusion matrix, I will import the necessary libraries (`confusion_matrix`, `matplotlib.pyplot`, `seaborn`), compute the confusion matrix using the `true_labels` and `predicted_labels`, and then visualize it as a heatmap with appropriate labels and a title.



In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix using seaborn heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

print("Confusion matrix calculated and plotted successfully.")

## Função para Aplicar o Modelo a um Novo Conjunto de Dados

### Subtask:
Criar uma função que aceite o caminho de um arquivo CSV, aplique o modelo FinBERT treinado para prever o sentimento de cada manchete, adicione as colunas 'Previsão_Sentimento' e 'Confiança' ao DataFrame e salve o resultado em 'previsoes_finais.csv'. Demonstrar o uso da função com um DataFrame de teste.

**Reasoning**:
I will define a Python function `apply_finbert_to_csv` that takes the input CSV path, reads it into a pandas DataFrame, and then iterates through the 'Text' column. For each text, it will call the already defined `analisar_mercado` function to get the predicted sentiment and confidence. These results will be stored in new columns 'Previsão_Sentimento' and 'Confiança'. Finally, the function will save the modified DataFrame to 'previsoes_finais.csv'. To demonstrate, I will create a sample DataFrame, save it as a temporary CSV, and then call the new function with this CSV.

In [ ]:
import pandas as pd
import torch

def apply_finbert_to_csv(input_csv_path, output_csv_path='previsoes_finais.csv'):
    """
    Aplica o modelo FinBERT a um conjunto de dados CSV, prevê o sentimento e a confiança,
    e salva o resultado em um novo arquivo CSV.

    Args:
        input_csv_path (str): Caminho para o arquivo CSV de entrada com uma coluna 'Text'.
        output_csv_path (str): Caminho para salvar o arquivo CSV de saída com as previsões.

    Returns:
        pandas.DataFrame: O DataFrame com as previsões adicionadas.
    """
    print(f"Lendo o arquivo CSV de entrada: {input_csv_path}")
    df_new = pd.read_csv(input_csv_path)

    # Certifica-se de que a coluna 'Text' existe
    if 'Text' not in df_new.columns:
        raise ValueError("O arquivo CSV de entrada deve conter uma coluna 'Text'.")

    # Inicializa as listas para armazenar as previsões
    predicted_sentiments = []
    confidences = []

    print("Aplicando o modelo FinBERT para prever sentimentos...")
    for index, row in df_new.iterrows():
        sentiment, confidence = analisar_mercado(row['Text'])
        predicted_sentiments.append(sentiment)
        confidences.append(confidence)

    # Adiciona as novas colunas ao DataFrame
    df_new['Previsão_Sentimento'] = predicted_sentiments
    df_new['Confiança'] = confidences

    # Salva o DataFrame resultante em um novo arquivo CSV
    df_new.to_csv(output_csv_path, index=False)
    print(f"Resultados salvos em: {output_csv_path}")

    return df_new

# --- Exemplo de Uso da Função ---

# 1. Criar um DataFrame de teste com novas manchetes
sample_data = {
    'Text': [
        "Tesla's stock price soared by 10% after announcing record-breaking deliveries.",
        "The company faced unexpected challenges leading to a significant drop in revenue.",
        "Analysts are neutral on the market outlook, awaiting further economic data.",
        "New product launch generated immense positive buzz and pre-orders."
    ]
}

df_sample_test = pd.DataFrame(sample_data)

# Salvar o DataFrame de teste em um CSV temporário para simular o arquivo de entrada
sample_input_csv = 'sample_news_headlines.csv'
df_sample_test.to_csv(sample_input_csv, index=False)
print(f"Arquivo de teste '{sample_input_csv}' criado com sucesso.")

# 2. Chamar a função para aplicar o modelo
final_predictions_df = apply_finbert_to_csv(sample_input_csv)

# 3. Exibir o DataFrame resultante
print("\nDataFrame final com previsões:")
display(final_predictions_df.head())


## Final Task

### Subtask:
Fornecer um resumo da avaliação do modelo, incluindo as métricas do relatório de classificação e as observações da matriz de confusão.


## Summary:

### Q&A
The model evaluation indicates an overall accuracy of 0.82. The 'Positive' class shows stronger performance with a precision of 0.85, recall of 0.87, and F1-score of 0.86. The 'Negative' class has slightly lower performance metrics, with a precision of 0.76, recall of 0.72, and F1-score of 0.74. The confusion matrix visually confirms these findings, showing that the model correctly classified more 'Positive' instances than 'Negative' instances.

### Data Analysis Key Findings
*   The model achieved an overall accuracy of 0.82 on the test dataset.
*   For the 'Negative' class (support 421): Precision is 0.76, Recall is 0.72, and F1-score is 0.74.
*   For the 'Positive' class (support 738): Precision is 0.85, Recall is 0.87, and F1-score is 0.86.
*   The confusion matrix revealed that out of 1159 instances, the model correctly identified 303 'Negative' instances (True Negatives) and 642 'Positive' instances (True Positives).
*   The model misclassified 118 'Negative' instances as 'Positive' (False Positives) and 96 'Positive' instances as 'Negative' (False Negatives).

### Insights or Next Steps
*   Given the imbalance in recall (0.72 for 'Negative' vs. 0.87 for 'Positive'), investigate methods to improve the model's ability to identify 'Negative' instances, such as adjusting class weights during training or exploring different loss functions.
*   Analyze the False Positives and False Negatives specifically to understand common characteristics of misclassified samples, which could inform feature engineering or data augmentation strategies.
